In [17]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [19]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)


In [20]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df = vine_df.withColumn("star_rating", 
                                  vine_df["star_rating"]
                                  .cast('integer'))

In [21]:
filtered_vine = vine_df.where(vine_df.total_votes>="20")

In [22]:
from pyspark.sql.functions import col
# import lit function
from pyspark.sql.functions import lit

In [35]:
#filtered_vine.withColumn("percent_helpful", filtered_vine.helpful_votes*5)\.show()

filtered_vine = filtered_vine.select("review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase", lit(filtered_vine.helpful_votes / filtered_vine.total_votes).alias("Percent_Helpful"))

In [37]:
filtered_vine = filtered_vine.where(filtered_vine.Percent_Helpful>=".5")

In [51]:
total_reviews = filtered_vine.select('review_id').count()
total_reviews

31830

In [41]:
verified_vine_df = filtered_vine.where(filtered_vine.vine=="Y")

In [59]:
verified_vine_count = verified_vine_df.count()
verified_vine_count

285

In [42]:
not_verified_vine_df = filtered_vine.where(filtered_vine.vine=="N")

In [66]:
not_verified_count = not_verified_vine_df.count()
not_verified_count

31545

In [46]:
verified_count = verified_vine_df.select('star_rating').where(verified_vine_df.star_rating=='5').count()
verified_count

163

In [65]:
not_verified_vine_count = not_verified_vine_df.select('star_rating').where(verified_vine_df.star_rating=='5').count()
not_verified_vine_count


14614

In [54]:
Percent_5_star_paid_allreviews = verified_count/total_reviews
Percent_5_star_paid_allreviews

0.00512095507382972

In [56]:
Percent_5_star_unpaid_allreviews = not_verified_vine_count/total_reviews
Percent_5_star_unpaid_allreviews

0.45912661011624256

In [60]:
Percent_5_star_paid = verified_count/verified_vine_count
Percent_5_star_paid

0.5719298245614035

In [67]:
Percent_5_star_unpaid =  not_verified_vine_count / not_verified_count
Percent_5_star_unpaid

0.46327468695514346